# Install Dependencies

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

# Script

In [ ]:
import time
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Configuração do Logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class InstagramScraper:
    def __init__(self, username, password):
        """Inicializa o WebDriver e define as credenciais do usuário."""
        self.username = username
        self.password = password
        self.driver = self._setup_driver()

    def _setup_driver(self):
        """Configura e retorna uma instância do WebDriver do Chrome."""
        options = Options()
        # options.add_argument("--headless") # Executar sem interface gráfica
        options.add_argument("--disable-gpu") # Desativa o uso da GPU para evitar possíveis problemas de renderização
        options.add_argument("--window-size=1920,1080") # Define o tamanho da janela do navegador para 1920x1080
        options.add_argument("--no-sandbox") # Desativa o modo sandbox para evitar restrições de execução (necessário em alguns ambientes)
        options.add_argument("--disable-dev-shm-usage") # Impede o uso de /dev/shm, útil para evitar erros em ambientes com pouca memória compartilhada
        
        service = Service(ChromeDriverManager().install())
        return webdriver.Chrome(service=service, options=options)

    def login(self):
        """Realiza login no Instagram."""
        logging.info("🔑 Acessando página de login...")
        self.driver.get("https://www.instagram.com/accounts/login/")

        try:
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "username"))).send_keys(self.username)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, "password"))).send_keys(self.password)
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))).click()
            time.sleep(5)  # Tempo para login

            # Verifica se login realmente foi aceito
            if "login" in self.driver.current_url:
                logging.error("❌ Parece que o login falhou! Verifique as credenciais.")
                self.driver.quit()
                return

            # Força um refresh na página para garantir que a sessão foi estabelecida
            self.driver.refresh()
            time.sleep(3)

            # Fecha pop-ups
            self.fechar_popups()

            logging.info("✅ Login bem-sucedido!")

        except Exception as e:
            logging.error(f"❌ Erro no login: {e}")
            self.driver.quit()
            raise

    def fechar_popups(self):
        """Fecha pop-ups de boas-vindas ou confirmações."""
        try:
            WebDriverWait(self.driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Agora não')]"))
            ).click()
            logging.info("🔄 Pop-up fechado!")
        except Exception:
            logging.info("⚠️ Nenhum pop-up encontrado.")
    
    def acessar_lista(self, tipo):
        """Navega até o perfil do usuário e clica na lista de seguidores ou seguidos."""
        perfil_url = f"https://www.instagram.com/{self.username}/"
        self.driver.get(perfil_url)
        logging.info(f"📌 Acessando perfil: {perfil_url}")

        xpath_lista = "//a[contains(@href, '/followers/')]" if tipo == "followers" else "//a[contains(@href, '/following/')]"

        try:
            botao_lista = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpath_lista))
            )
            botao_lista.click()
            logging.info(f"📂 Abrindo lista de {tipo}...")
        except Exception as e:
            logging.error(f"❌ Erro ao tentar abrir a lista de {tipo}: {e}")
            self.driver.quit()
            raise

    def rolar_modal(self):
        """Rola a lista de seguidores ou seguidos para carregar todos os itens."""
        logging.info("⏳ Aguardando abertura do modal...")

        modal_xpath = "//div[contains(@class, 'xyi19xy x1ccrb07 xtf3nb5 x1pc53ja x1lliihq x1iyjqo2 xs83m0k xz65tgg x1rife3k x1n2onr6')]"

        try:
            modal = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, modal_xpath))
            )
        except Exception as e:
            logging.error(f"❌ Não foi possível encontrar o modal: {e}")
            self.driver.quit()
            raise

        # Obtém a altura inicial do modal para acompanhar o progresso da rolagem
        last_height = self.driver.execute_script("return arguments[0].scrollHeight", modal)
        tentativas_sem_atualizacao = 0 # Contador para detectar quando não há mais novos itens

        while tentativas_sem_atualizacao < 3:
            self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", modal)
            time.sleep(2)  # Tempo para carregar mais itens

            # Obtém a nova altura do modal após a rolagem
            new_height = self.driver.execute_script("return arguments[0].scrollHeight", modal)

            if new_height == last_height:
                tentativas_sem_atualizacao += 1
            else:
                tentativas_sem_atualizacao = 0

            last_height = new_height
            logging.info("🔄 Rolando a lista para carregar mais itens...")

        logging.info("✅ Rolagem concluída!")

    def obter_lista(self, tipo):
        """Obtém e retorna a lista de seguidores ou seguidos."""
        lista_xpath = "//div[contains(@class, 'x1dm5mii x16mil14 xiojian x1yutycm x1lliihq x193iq5w xh8yej3')]"

        time.sleep(2)
        elementos = self.driver.find_elements(By.XPATH, lista_xpath)

        lista = set()
        for el in elementos:
            partes = el.text.strip().split("\n") # Remove espaços extras e divide o texto do elemento em linhas com base nas quebras de linha
            if partes:
                lista.add(partes[0])  # Primeiro item geralmente é o nome de usuário

        logging.info(f"✅ {tipo.capitalize()} encontrados: {len(lista)}")
        return list(lista)

    def executar(self):
        """Executa todo o processo de login, captura e comparação de listas."""
        try:
            self.login()

            # Captura a lista de seguidores
            self.acessar_lista("followers")
            self.rolar_modal()
            seguidores = self.obter_lista("seguidores")

            # Captura a lista de seguidos
            self.acessar_lista("following")
            self.rolar_modal()
            seguindo = self.obter_lista("seguidos")

            # Identifica quem não segue de volta
            nao_seguem_de_volta = sorted([user for user in seguindo if user not in seguidores])

            # Exibe um resumo dos resultados
            logging.info("\n📊 Resumo da análise:")
            logging.info(f"👥 Seguidores: {len(seguidores)}")
            logging.info(f"➡️ Seguindo: {len(seguindo)}")
            logging.info(f"❌ Não seguem de volta: {len(nao_seguem_de_volta)}\n")

            return seguidores, seguindo, nao_seguem_de_volta

        finally:
            self.driver.quit()
            logging.info("🛑 Navegador fechado.")

# Executando o código
if __name__ == "__main__":
    USERNAME = "seu_username_aqui" 
    PASSWORD = "sua_senha_aqui"

    scraper = InstagramScraper(USERNAME, PASSWORD)
    seguidores, seguindo, nao_seguem_de_volta = scraper.executar()
